# 16.413 Final Project: Convex Risk Bounded Continuous-Time Trajectory Planning for a Europa Lander

Julia Briden, Kota Kondo, Lakshay Sharma, Nick Rober, and Youngjae Min

## 1. Problem Formulation

Github code from paper: https://github.com/jasour/Risk-Bounded-Continuous-Time-Trajectory-Planning/blob/main/RRT_Tools/RRT_SOS_Static.m


Determine obstacles (dynamic or static, convex or nonconvex), start and goal points, probabilistic optimization problem formulation, constraints, planning time horizon

https://trs.jpl.nasa.gov/bitstream/handle/2014/47541/CL%2317-0518.pdf?sequence=1

Precision Landing Maneuver:
Upon receiving an initial surface-relative position and velocity estimate from the ILS, guidance
will perform the Precision Landing Maneuver (PLM). The objective of this maneuver is to steer
the PDV so that, at the end of the maneuver, the PDV is vertically-oriented directly above the
landing site at an altitude of 1000 m with zero horizontal velocity and vertical velocity of -30 m/s.
When the PLM is complete, the PDV will descend to an altitude of 500 m to begin Hazard
Detection and Avoidance.

Hazard Detection and Avoidance:
When the PDV is at an altitude of 500 m and directly above the targeted landing location, it uses
the 3D-imaging lidar to scan the landing site, construct a safe landing map, and select a landing 
Pre-decisional: for information and discussion only 13
location within that map. This activity must complete in three seconds, at which time the PDV will
be approximately 400 m above the targeted landing location.
Once the Hazard Detection activity has finished, guidance uses the new, safe landing location
to construct a guidance profile and steer the vehicle to a point above the targeted safe landing spot.
The Hazard Avoidance Maneuver begins at an altitude of 250 m and completes at an altitude of
30 m with a vertical velocity of -0.5 m/s. The safe landing map, using the data from the 3D-imaging
lidar, will be 100 m  100 m and the vehicle has the capability to land anywhere within that map.

![alt text](europalander.png "Europa Lander mission concept")

## 2. Risk Contours

Risk contours are defined as the set of all points in uncertain environments with guaranteed bounded risk. We will use risk contours in finding optimal trajectories. 

For simplicity we incorporate only static risk contours into our problem, not dynamic contours, and replace probabilistic constraint with deterministic constraint in terms of x (use inner approximation of probabilisitic constraint to bound risk), which lead to rational polynomial representation.

### How to construct risk contours

Define $\chi \in \mathbb{R}^2$ as an uncertain environment where our ground robot explores and $\chi_{obs_{i}}(\omega_i) \subset \chi$, for $i = 1, ..., n_{o_{s}}$ as static uncertain obstacles, where $n_{o_{s}}$ is the number of static obstacles, $\omega_i$ is a probablistic uncertain parameters. 

Then, we can represent $\chi_{obs_{i}}(\omega_i)$ in terms of polynomials in x $\in \chi$

***
$\chi_{obs_{i}}(\omega_i) = \{x \in \chi | P_i(x, \omega_i) \geq 0\}$ for i = 1, ..., $n_{o_{s}}$ (1)
***

where P_i : $\mathbb{R}^{n_x + n_{\omega}} \to \mathbb{R}$ denotes a given polynomial. Let $\triangle \in [0, 1]$ be a given acceptable risk level, and then, $C^{\triangle}_{r}$, static risk contour whose risk level, with less than the given acceptable can be defined as

***
$C^{\triangle}_{r} \equiv \{x \in \chi | Prob(x \in \chi_{obs}(\omega)) \leq \triangle\}$ (2)
***

By this definition, static risk contour is now defined with a deterministic constraint in terms of x. Furthermore, the statci risk contour's inner approximation is given as

***
$\hat{C}^{\triangle}_{r} \equiv \{x  \in \chi | \frac{\mathbb{E}[P^2(x, \omega)] - \mathbb{E}[P(x, \omega)]^2}{\mathbb{E}[P^2(x, \omega)]} \leq \triangle  or  \mathbb{E}[P(x, \omega)] \leq 0 \}$ (3)
***

Since $\hat{C}^{\triangle}_{r}$ is an approximation of inner space of $C^{\triangle}_{r}$, any trajectory going through $\hat{C}^{\triangle}_{r}$ has a rsik less or equal to the given $\triangle$.

*Convex Risk Bounded Continuous-Time Trajectory Planning in Uncertain Nonconvex Environments (https://arxiv.org/pdf/2106.05489.pdf)* gives a good expample to illustrate the use of this risk contour generation method.

### Function outline for generating random obstacles in the environment

In [25]:
## Function definition
#
# input
#    env: environment size [x1, y1, x2, y2]
#    shape: obstacle shape options (eg. circle, rectangle, etc)
#.   max_size: maximum obstacle size [x_max, y_max]
#    omega_range: range of potential probablistic uncertain parameters [min, max]
#
# output
#    pols: a list of polynomials that represents an obstacle, including probabilistic uncertain parameter, omega
#

def random_ob(env, shape, max_size, omega_range):
    # randomly generate obstacles within the given environments
    # return a list of polynomials generated according to the given shape, maximum size, and omega range
    return pols

### Function outline for static risk contours 

In [26]:
## Function definition
#
# input
#    delta: acceptable risk level, [0,1]
#    pol: polynomial that represents an obstacle, including probabilistic uncertain parameter, omega  
#
# output
#    C: the boundary of inner approximation of risk contour, which is denoted as a polynomial
#

def static_rc(delta, pol):
    # calculate C using equation (3)
    # need to calculate expected values using the given polynomials
    # since delta is given, we can find the boundary of C as a form of a polynomial
    # may need another function to convert the given polynomial to another form. 
    return C

## 3. Continuous-Time Risk Bounded Trajectory Planning using Risk Contours

https://arxiv.org/pdf/2106.05489.pdf

1. Transform the probabilistic optimization problem into a deterministic polynomial optimization problem -> nonconvex and nonlinear. Must ensure all deterministic constraints are satisfied over the entire planning time horizon.

a. Replace stochastic constraints with deterministic constraints

b. Substitute polynomial trajectory in optimization problem -> deterministic optimization with constraints in terms of the coefficients of the polynomial trajectory x(t)

2. Convex methods based on SOS techniques to solve the nonconvex time-varying deterministic planning optimization problem (no time discretization required for guarentees)

RRT-SOS: sampling-based motion planning to construct the risk bounded trajectory of the deterministic polynomial optimization. Use an SOS-based continuous-time technique to verify the constraints of the optimization -> use SOS polynomials for x(t) (complexity is independent of the size of planning time horizon and length of polynomial trajectory. Use RRT* algorithm and include additional constraint formulations (line of sight, glide slope, spacecraft tilt angle, angle of attack, rate constraints, state-triggered constraints, time-triggered constraints, control constraints) to extend research paper's work. This algorithm is slower but always returns a feasible trajectory.

a. Use RRT*, with smart initialization to improve runtime, (extension of paper's naive RRT formulation) trajectory to connect the given sample point to the closest vertex in the tree, if the trajectory satisfies the SOS condition (inside risk contours).

b. If constraints are satisfied, connect the sample point to the goal point (iterate to get a feasible trajectory between start and goal states).

c. Improve feasible trajectory / tree

   i. construct a graph (eg PRM) whose nodes are the vertex of the tree and edges are all trajecoties between nodes that satisfy the SOS condition
    
   ii. perform a shortest path algorithm (eg Dijkstra) to obtain a path from start to goal point